In [26]:
using Pkg
Pkg.activate(joinpath(pwd(),".."))
using Random, Distributions, PrettyTables, Printf, Plots, Optim
Random.seed!(1234)

  Activating environment at `~/Desktop/ECON 627/Project.toml`


MersenneTwister(1234)

In [27]:
function data(n)
    mvnormal = MvNormal([0;0], [1 0.99 ; 0.99 1])
    err = rand(mvnormal, n)'

    U = err[:,1]
    V = err[:,2]

    X2 = randn(n)
    Z = randn(n)
    X1 = Z + X2 + V
    Y = sin.(X1 + X2) + U

    return (Y = Y, X1 = X1, X2 = X2, Z = Z)
end

data (generic function with 1 method)

In [89]:
function Qn(gamma, Y, X1, X2)
    n = length(Y)
    Qn = 0
    gamma = gamma[1]
    for i in 1:n
        Yi = Y[i]
        X1i = X1[i]
        X2i = X2[i]
        gi = sin(gamma * X1i + X2i)
        Qn += 1/n * (Yi - gi)^2
    end
    return Qn = Qn
end


Qn (generic function with 1 method)

In [57]:
function FV(Y, X1, X2, Z)
    W = [Z X2]
    beta = inv(W'*W)*W'*X1
    X1_hat = W * beta
    
    result = optimize(gamma -> Qn(gamma,Y,X1_hat, X2),[0.0;])
    gamma_FV = Optim.minimizer(result)
    return gamma_FV = gamma_FV
end

FV (generic function with 2 methods)

In [90]:
function Qn2(gamma, Y, X1, X2, V)
    gamma1 = gamma[1]
    gamma2 = gamma[2]

    n = length(Y)
    Qn2 = 0

    for i in 1:n
        Yi = Y[i]
        X1i = X1[i]
        X2i = X2[i]
        Vi = V[i]
        gi = sin(gamma1 * X1i + X2i)
        Qn2 += 1/n * (Yi - gi - gamma2 * Vi)^2
    end
    return Qn2 = Qn2
end

Qn2 (generic function with 1 method)

In [82]:
function CF(Y, X1, X2, Z)
    W = [Z X2]
    beta = inv(W'*W)*W'*X1
    X1_hat = W * beta
    V_hat = X1 - X1_hat

    result = optimize(gamma -> Qn2(gamma,Y,X1, X2, V_hat),[0.0;0.0;])
    gamma_CF = Optim.minimizer(result)
    return gamma_CF = gamma_CF
end

    

CF (generic function with 2 methods)

In [111]:
function result(n)
    bias_FV = 0
    bias_CF = 0
    RMSE_FV = 0
    RMSE_CF = 0

    for r in 1:1000
        (Y, X1, X2, Z) = data(n)
        gamma_FV = FV(Y, X1, X2, Z)
        gamma_CF = CF(Y, X1, X2, Z)
        bias_FV += 1/1000 * (gamma_FV[1] - 1)
        bias_CF += 1/1000 * (gamma_CF[1] - 1)
        RMSE_FV += 1/1000 * (gamma_FV[1] - 1)^2
        RMSE_CF += 1/1000 * (gamma_CF[1] - 1)^2
    end


    println("The mean bias for fitted value estimator is ", bias_FV)
    println("The mean bias for control function estimator is ", bias_CF)
    println("The RMSE for fitted value estimator is ", RMSE_FV)
    println("The RMSE for control function estimator is ", RMSE_CF)
end

result (generic function with 1 method)

# Part (a): 100 samples

In [116]:
result(100)

The mean bias for fitted value estimator is -0.25952934722900367
The mean bias for control function estimator is -0.0005794762123971724
The RMSE for fitted value estimator is 0.3916889048078497
The RMSE for control function estimator is 0.0033243569456523152


# Part (b): 1000 samples

In [118]:
result(1000)

The mean bias for fitted value estimator is -0.003774085998534703
The mean bias for control function estimator is 0.0005871224719415906
The RMSE for fitted value estimator is 0.012167836930730855
The RMSE for control function estimator is 8.386969371484683e-5


# Part (c)

From the previous result, we can see that the biases of both estimators are smaller when we have larger sample size, meaning they can both be consistent. However, compared with fitted value estimator, control function estimator has lower bias and RMSE, and is more appropriate to be the estimator for nonlinear IV models.